https://www.kaggle.com/sourabhy/face-mask-detection-using-transfer-learning

In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [28]:
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D,Flatten, Reshape, Bidirectional, LSTM, Dense, Lambda, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam,SGD
import glob
import tensorflow as tf
import shutil

In [31]:
with_maskImage = os.listdir('data/with_mask')
without_maskImage = os.listdir('data/without_mask')

In [32]:
os.mkdir('./data/train')
os.mkdir('./data/train/with_mask')
os.mkdir('./data/val')
os.mkdir('./data/val/with_mask')
os.mkdir('./data/test')
os.mkdir('./data/test/with_mask')


os.mkdir('./data/train/without_mask')
os.mkdir('./data/val/without_mask')
os.mkdir('./data/test/without_mask')

In [34]:
with_mask_train_len = int(np.round(0.6 * len(with_maskImage), 0))
with_mask_val_len = int(np.round(0.7 * len(with_maskImage), 0))


for i in range(with_mask_train_len):
    shutil.copy(os.path.join('data/with_mask', with_maskImage[i]), './data/train/with_mask')

for i in range(with_mask_train_len, with_mask_val_len):
    shutil.copy(os.path.join('data/with_mask', with_maskImage[i]), './data/val/with_mask')

for i in range(with_mask_val_len, len(with_maskImage)):
    shutil.copy(os.path.join('data/with_mask', with_maskImage[i]), './data/test/with_mask')
    

without_mask_train_len = int(np.round(0.6 * len(without_maskImage), 0))
without_mask_val_len = int(np.round(0.7 * len(without_maskImage), 0))



for i in range(without_mask_train_len):
    shutil.copy(os.path.join('data/without_mask', without_maskImage[i]), './data/train/without_mask')

for i in range(without_mask_train_len, without_mask_val_len):
    shutil.copy(os.path.join('data/without_mask', without_maskImage[i]), './data/val/without_mask')

for i in range(without_mask_val_len, len(without_maskImage)):
    shutil.copy(os.path.join('data/without_mask', without_maskImage[i]), './data/test/without_mask')

In [35]:
PATH_TRAIN_MASK = './data/train/with_mask'
PATH_VAL_MASK = './data/val/with_mask'
PATH_TEST_MASK = './test/with_mask'

PATH_TRAIN_NO_MASK = './data/train/without_mask'
PATH_VAL_NO_MASK = './data/val/without_mask'
PATH_TEST_NO_MASK = './data/test/without_mask'

PATH_TRAIN = './data/train'
PATH_VAL = './data/val'
PATH_TEST = './data/test'

In [36]:
train_mask_files = os.listdir(PATH_TRAIN_MASK)
train_no_mask_files = os.listdir(PATH_TRAIN_NO_MASK)

In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img

train_data_gen = ImageDataGenerator(rotation_range=30,
                                    width_shift_range=0.02,
                                    height_shift_range=0.02,
                                    zoom_range=[0.8,1.2],
                                    horizontal_flip=True,
                                    rescale=1/255
                                   )


val_data_gen = ImageDataGenerator(rescale=1/255)
test_data_gen = ImageDataGenerator(rescale=1/255)

In [38]:
training_data = train_data_gen.flow_from_directory(PATH_TRAIN, target_size=(200, 200), color_mode='rgb', class_mode='binary', batch_size=32)
val_data = val_data_gen.flow_from_directory(PATH_VAL, target_size=(200, 200), color_mode='rgb', class_mode='binary', batch_size=32)
test_data = test_data_gen.flow_from_directory(PATH_TEST, target_size=(200, 200), color_mode='rgb', class_mode='binary', batch_size=32)

Found 4532 images belonging to 2 classes.
Found 756 images belonging to 2 classes.
Found 2265 images belonging to 2 classes.


In [39]:
training_data.class_indices

{'with_mask': 0, 'without_mask': 1}

In [41]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [42]:
md=ResNet50(include_top=False,input_shape=(200, 200, 3))

94773248/94765736 [==============================] - 18s 0us/step


In [43]:
inputl=Input(shape=(200, 200, 3), name='input')
l1=md(inputl)
l2=Flatten()(l1)
l4=Dropout(0.3)
l3=Dense(10,activation='relu')(l2)
l4=Dropout(0.3)
output=Dense(1,activation='sigmoid')(l3)

model=Model(inputs=[inputl],outputs=output)

In [44]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 200, 200, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 10)                1003530   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 24,591,253
Trainable params: 24,538,133
Non-trainable params: 53,120
_________________________________________________________________


In [46]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [48]:
history = model.fit(training_data, batch_size=96, epochs=10, validation_data=val_data)

Epoch 1/10
  8/142 [>.............................] - ETA: 26:41 - loss: 0.8582 - acc: 0.8594 - f1_m: 0.8640 - precision_m: 0.8242 - recall_m: 0.9330

KeyboardInterrupt: 

In [49]:
model.evaluate(test_data)

 5/71 [=>............................] - ETA: 2:11 - loss: 187.2149 - acc: 0.5188 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    
    f1_list = [s for s in history.history.keys() if 'f1_m' in s and 'val' not in s]
    val_f1_list = [s for s in history.history.keys() if 'f1_m' in s and 'val' in s]
    
    precision_m_list = [s for s in history.history.keys() if 'precision_m' in s and 'val' not in s]
    val_precision_m_list = [s for s in history.history.keys() if 'precision_m' in s and 'val' in s]
    
    recall_m_list = [s for s in history.history.keys() if 'recall_m' in s and 'val' not in s]
    val_recall_m_list = [s for s in history.history.keys() if 'recall_m' in s and 'val' in s]
    
    
    
    
    
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
    
    
    
    ## F1 Score
    plt.figure(3)
    for l in f1_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_f1_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('F1 score')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
    
   ## Precision Score
    plt.figure(4)
    for l in precision_m_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_precision_m_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Precision score')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show() 
    
    
     ## Precision Score
    plt.figure(5)
    for l in recall_m_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_recall_m_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Recall')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show() 

In [ ]:
plot_history(history)

In [ ]:
def full_multiclass_report(model,
                           x,
                           y_true,
                           batch_size=32,
                           binary=False):

    
    loss,acc,F1, precision, recall=model.evaluate(x)
    
    
    # 3. Print accuracy score
    print("Accuracy : "+ str(acc))
    print("F1 score : "+ str(F1))
    print("Precision : "+ str(precision))
    print("Recall : "+ str(recall))

In [ ]:
full_multiclass_report(model,
                       test_data,
                       test_data.classes)

In [ ]:
full_multiclass_report(model,
                      val_data,
                       val_data.classes)